In [1]:
import pickle
import numpy as np
import deepchem as dc
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
def featurizer(featname, data, plant):
    if featname == "MACCS":
        featurizer_maccs = dc.feat.MACCSKeysFingerprint()
        data_features = featurizer_maccs.featurize(data['smiles'])
        plant_features = featurizer_maccs.featurize(plant['SMILES'])

        vt = VarianceThreshold(threshold=(.98 * (1 - .98)))
        data_selected = vt.fit_transform(data_features)
        mask = vt.get_support(indices=True)
        plant_selected = plant_features[:, mask]
        dataset = dc.data.NumpyDataset(data_selected, data['class'])
    
    elif featname == "RDkit":
        featurizer_rdkit = dc.feat.RDKitDescriptors()
        data_features = featurizer_rdkit.featurize(data['smiles'])
        plant_features = featurizer_rdkit.featurize(plant['SMILES'])

        vt = VarianceThreshold(threshold=(.98 * (1 - .98)))
        data_selected = vt.fit_transform(data_features)
        mask = vt.get_support(indices=True)
        plant_selected = plant_features[:, mask]
        dataset = dc.data.NumpyDataset(data_selected, data['class'])
    
    else:
        raise ValueError("Unsupported feature type")

    return dataset, plant_selected

In [ ]:
data = pd.read_csv('../data/GABAA.csv',encoding='gb18030')
new_data = pd.read_excel('../data/application_domain.xlsx',sheet_name='Removes rdkit error SMILES')

new_maccs, plant_maccs = featurizer('MACCS', data, new_data)
new_rdkit, plant_rdkit = featurizer('RDkit', data, new_data)